<img src='conv.png' width=80%>
<img src='gnn.png' width=80%>

In [1]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class Loader:
    def __init__(self):
        pass
    
    def load_dataset(self):    
        
        '''
        citations: [target논문인덱스, source논문인덱스]
        papers: [논문인덱스, 1424개 단어 포함 여부, 주제(subject)]
        train_data: papers 데이터 중 50% 샘플링
        test_data: papers 데이터 중 50% 샘플링
        x_train: train_data 중, 논문인덱스와 subject를 제외한 피쳐
        y_train: train_data 중 subject에 해당하는 레이블
        '''
        
        zip_file = keras.utils.get_file(
            fname="cora.tgz",
            origin="https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz",
            extract=True,
        )
        data_dir = os.path.join(os.path.dirname(zip_file), "cora")
        
        citations = pd.read_csv(
            os.path.join(data_dir, "cora.cites"),
            sep="\t",
            header=None,
            names=["target", "source"],
        )
        
        column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
        papers = pd.read_csv(
            os.path.join(data_dir, "cora.content"), 
            sep="\t", 
            header=None, 
            names=column_names,
        )
        
        class_values = sorted(papers["subject"].unique())
        class_idx = {name: id for id, name in enumerate(class_values)}
        paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

        papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
        citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
        citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
        papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])
        
        train_data, test_data = [], []

        for _, group_data in papers.groupby("subject"):
            # Select around 50% of the dataset for training.
            random_selection = np.random.rand(len(group_data.index)) <= 0.5
            train_data.append(group_data[random_selection])
            test_data.append(group_data[~random_selection])

        train_data = pd.concat(train_data).sample(frac=1)
        test_data = pd.concat(test_data).sample(frac=1)

        print("citations data shape:", citations.shape)
        print("papers data shape:", papers.shape)
        print("Train data shape:", train_data.shape)
        print("Test data shape:", test_data.shape)
        
        feature_names = set(papers.columns) - {"paper_id", "subject"}
        num_features = len(feature_names)
        num_classes = len(class_idx)

        # Create train and test features as a numpy array.
        x_train = train_data[feature_names].to_numpy()
        x_test = test_data[feature_names].to_numpy()
        
        # Create train and test targets as a numpy array.
        y_train = train_data["subject"]
        y_test = test_data["subject"]
        
        # Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
        edges = citations[["source", "target"]].to_numpy().T
        # Create an edge weights array of ones.
        edge_weights = tf.ones(shape=edges.shape[1])
        # Create a node features array of shape [num_nodes, num_features].
        node_features = tf.cast(
            papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
        )
        # Create graph info tuple with node_features, edges, and edge_weights.
        graph_info = (node_features, edges, edge_weights)

        print("Edges shape:", edges.shape)
        print("Nodes shape:", node_features.shape)
        
        self.feature_names = feature_names #단어 피쳐 컬럼 이름
        self.num_features = num_features #단어 피쳐 개수
        self.num_classes = num_classes #주제subject의 가짓수
        self.graph_info = graph_info
        self.node_features = node_features #논문 인덱스 순으로 정리된 상태의 feature 매트릭스
        self.edges = edges
        self.class_values = class_values
        
        return citations, papers, x_train, x_test, y_train, y_test, train_data, test_data
    
    

In [3]:
loader = Loader()
citations, papers, x_train, x_test, y_train, y_test, train_data, test_data = loader.load_dataset()
feature_names = loader.feature_names
num_classes = loader.num_classes
graph_info = loader.graph_info

citations data shape: (5429, 2)
papers data shape: (2708, 1435)
Train data shape: (1378, 1435)
Test data shape: (1330, 1435)
Edges shape: (2, 5429)
Nodes shape: (2708, 1433)


In [4]:
def create_ffn(hidden_units, dropout_rate, name=None):
    '''
hidden_units = [32, 32]
dropout_rate = 0.2
'''

    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

In [5]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GraphConvLayer, self).__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        num_nodes = tf.math.reduce_max(node_indices) + 1
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """

        node_repesentations, edges, edge_weights = inputs
        # Get node_indices (source) and neighbour_indices (target) from edges.
        node_indices, neighbour_indices = edges[0], edges[1]
        # neighbour_repesentations shape is [num_edges, representation_dim].
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

        # Prepare the messages of the neighbours.
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(node_indices, neighbour_messages)
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)

In [6]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(GNNNodeClassifier, self).__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        # Compute logits
        return self.compute_logits(node_embeddings)

In [7]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 100
batch_size = 256

gnn_model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

print("GNN output shape:", gnn_model([1, 10, 100]))

gnn_model.summary()

GNN output shape: tf.Tensor(
[[ 0.01042217  0.1271193  -0.04811429  0.10840625  0.08604226 -0.10460234
   0.04044523]
 [ 0.10392918 -0.06908441 -0.06521179  0.05655614  0.02580935 -0.09190933
  -0.04335675]
 [ 0.12119567 -0.09860111 -0.04154585  0.08767194 -0.0387424   0.04388608
   0.01892218]], shape=(3, 7), dtype=float32)
Model: "gnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocess (Sequential)     (2708, 32)                52804     
                                                                 
 graph_conv1 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                                 
 graph_conv2 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                            

In [8]:


def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.15,
        callbacks=[early_stopping],
    )

    return history

In [9]:
x_train = train_data.paper_id.to_numpy()
history = run_experiment(gnn_model, x_train, y_train)

Epoch 1/100
5/5 [==============================] - 3s 173ms/step - loss: 2.1804 - acc: 0.1716 - val_loss: 1.9168 - val_acc: 0.3043
Epoch 2/100
5/5 [==============================] - 0s 74ms/step - loss: 1.9494 - acc: 0.2878 - val_loss: 1.9069 - val_acc: 0.1643
Epoch 3/100
5/5 [==============================] - 0s 75ms/step - loss: 1.9067 - acc: 0.2886 - val_loss: 1.8975 - val_acc: 0.1787
Epoch 4/100
5/5 [==============================] - 0s 79ms/step - loss: 1.8641 - acc: 0.2972 - val_loss: 1.8819 - val_acc: 0.3816
Epoch 5/100
5/5 [==============================] - 0s 73ms/step - loss: 1.8383 - acc: 0.3091 - val_loss: 1.8681 - val_acc: 0.3430
Epoch 6/100
5/5 [==============================] - 0s 81ms/step - loss: 1.8254 - acc: 0.3057 - val_loss: 1.8437 - val_acc: 0.3188
Epoch 7/100
5/5 [==============================] - 0s 83ms/step - loss: 1.8126 - acc: 0.3228 - val_loss: 1.8179 - val_acc: 0.3333
Epoch 8/100
5/5 [==============================] - 0s 78ms/step - loss: 1.7822 - acc: 0.3

In [16]:
gnn_model

In [18]:
test_data.paper_id.iloc[:10]

1239    2408
551      706
250      509
139     1642
417     1230
1146     821
836     2600
64      1374
974      877
2601    1982
Name: paper_id, dtype: int64

In [19]:
gnn_model.predict(test_data.paper_id.iloc[:10])

array([[[-4.344487  , -1.9264449 ,  7.687362  ,  0.74706805,
         -2.191979  , -1.5481755 , -0.07419495]],

       [[ 0.22949243, -1.526889  ,  0.50571024, -0.83466494,
         -0.26296428,  2.1523473 ,  0.433302  ]],

       [[-3.7569265 , -4.457555  ,  2.7660267 ,  7.8000865 ,
         -4.072202  , -1.5268492 , -1.3989537 ]],

       [[-1.623553  ,  6.437792  ,  0.952935  , -2.0380113 ,
          1.8217608 , -4.108915  ,  0.6353894 ]],

       [[ 0.8852041 , -1.4079202 ,  1.104238  , -0.45888564,
         -2.7183044 ,  1.1600124 ,  3.7676055 ]],

       [[-1.1969954 , -0.98133934,  3.1646774 ,  0.59719115,
         -0.62568814, -0.846568  , -0.16888843]],

       [[-0.87773895, -1.2683634 ,  3.1936808 ,  0.14415199,
         -1.983511  , -0.04533061,  1.5278798 ]],

       [[ 4.840487  , -5.6789594 , -1.6817155 , -1.4062569 ,
         -2.3526888 ,  0.28257075,  0.64046234]],

       [[-1.2520379 ,  5.9443674 ,  1.410282  , -1.4646583 ,
          0.7471036 , -2.8317802 ,  3.09544

In [10]:
x_test = test_data.paper_id.to_numpy()
_, test_accuracy = gnn_model.evaluate(x=x_test, y=y_test, verbose=0)
print(f"Test accuracy: {round(test_accuracy * 100, 2)}%")

Test accuracy: 77.07%


In [11]:
# Prediction

In [12]:
class_values = loader.class_values
node_features=loader.node_features
edges = loader.edges

def display_class_probabilities(probabilities):
    for instance_idx, probs in enumerate(probabilities):
        print(f"Instance {instance_idx + 1}:")
        for class_idx, prob in enumerate(probs):
            print(f"- {class_values[class_idx]}: {round(prob * 100, 2)}%")
            


In [13]:
logits = gnn_model.predict(tf.constant([2708, 2709, 2710]))
# logits = gnn_model.predict(tf.convert_to_tensor(new_node_indices[:2]))
probabilities = keras.activations.softmax(tf.convert_to_tensor(logits)).numpy()
display_class_probabilities(probabilities)

InvalidArgumentError:  indices[0] = 2708 is not in [0, 2708)
	 [[node gnn_model/GatherV2
 (defined at C:\Users\USER-PC\AppData\Local\Temp/ipykernel_23844/2180048713.py:66)
]] [Op:__inference_predict_function_15899]

Errors may have originated from an input operation.
Input Source operations connected to node gnn_model/GatherV2:
In[0] gnn_model/postprocess/dense_11/Gelu/mul_1 (defined at C:\Users\USER-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\layers\core\dense.py:213)	
In[1] IteratorGetNext (defined at C:\Users\USER-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1610)	
In[2] gnn_model/GatherV2/axis:

Operation defined at: (most recent call last)
>>>   File "C:\Users\USER-PC\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\USER-PC\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
>>>     